<h1><u>"Desafío 1: Análisis exploratorio de un dataset de precios de propiedades"</h1></u>
<br>
<h2>Grupo N°2 - Integrantes:</h2>
<ul>
  <li>Torres Obando, Juan Manuel</li>
  <li>Volman, Uriel brian</li>
  <li>Aguirre, Rodrigo</li>
  <li>González Pluchino, Leandro</li>
</ul>

![Imagen](https://www.academiadeinversion.com/wp-content/uploads/2014/03/como-calcular-valor-inmueble-metodos-ejemplos.png)

**Montaje para la unidad en Google Drive**

Importación de las librerías

In [1]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import plotly as pl
import plotly.graph_objects as go
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
pl.offline.init_notebook_mode(connected=True)
pd.set_option('display.float_format', lambda x: '%.9f' % x)
pd.options.display.float_format = '{:,}'.format

Importación del Conjunto de datos

In [2]:
#Creamos el dataframe con la base de Properatti
path=r"C:\Users\rodri\Desktop\Data Science\properatti.csv"
df=pd.read_csv(path)
df.head(40)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"3,430,787.0","-34.6618237,-58.5088387",-34.6618237,...,40.0,"1,127.2727272727273","1,550.0",nan,nan,nan,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,"3,432,039.0","-34.9038831,-57.9643295",-34.9038831,...,nan,nan,nan,nan,nan,nan,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"3,430,787.0","-34.6522615,-58.5229825",-34.6522615,...,55.0,"1,309.090909090909","1,309.090909090909",nan,nan,nan,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,"3,431,333.0","-34.6477969,-58.5164244",-34.6477969,...,nan,nan,nan,nan,nan,nan,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,"3,435,548.0","-38.0026256,-57.5494468",-38.0026256,...,35.0,"1,828.5714285714291","1,828.5714285714291",nan,nan,nan,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
5,5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,"3,433,657.0","-33.0140714,-58.519828",-33.0140714,...,nan,nan,nan,nan,nan,nan,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...
6,6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,"3,430,511.0","-34.5329567,-58.5217825",-34.53295670000001,...,78.0,"1,226.4150943396226","1,666.666666666667",nan,nan,nan,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"3,436,077.0","-34.5598729,-58.443362",-34.5598729,...,40.0,"3,066.666666666666","3,450.0",nan,nan,nan,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"3,436,077.0","-34.5598729,-58.443362",-34.5598729,...,60.0,"3,000.0","3,250.0",nan,nan,nan,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
9,9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,"3,838,574.0","-32.942031,-60.7259192",-32.942031,...,nan,nan,nan,nan,nan,nan,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 

In [3]:
#Vemos las dimensiones de la base con el metodo shape
#filas=121.220
#columnas=26
df.shape

(121220, 26)

In [4]:
#Listamos los campos del dataframe
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [5]:
#Renombramos los campos
dic_reemplazo={'property_type':'Tipo de propiedad',
               'place_name':'Partido',
               'state_name':'Provincia',
               "price":"Precio Aviso",
               "currency":"Moneda Aviso",
               "surface_covered_in_m2":"Superfecie Cubierta",
               "price_usd_per_m2":"Precio M2 USD",
               "price_per_m2":"Precio M2",
               "floor":"Piso",
               "expenses":"Expensas",
               "rooms":"Ambientes",
               "description":"Descripcion",
               "title":"Título",
               "price_aprox_local_currency":"Precio ML",
               "surface_total_in_m2":"Superficie Total",
               "price_aprox_usd":"Precio USD",
               "Unnamed: 0":"ID de registro",
               "lat":"Latitud",
               "lon":"Longitud",
               "lat-lon": "LatLon",
               "operation":"Operacion",
               "country_name":"Pais"}

df.rename(columns = dic_reemplazo, inplace = True)

#Eliminamos las columnas inncesarias:


In [6]:
#Demostramos que "Unnamed: 0" es igual al indice por lo que resulta redundante
df["index"]=list(df.index) #creamos una nueva columna que contiene el index
(df["index"]==df["ID de registro"]).value_counts()[1] #comparamos "index" con "Unnamed: 0", lo que genera una lista de booleanos
                                                  #y luego contamos los valores verdaderos. Devuelve 121.220 Trues,
                                                  #vemos que hay match en todos los registros de las columnas

121220

In [7]:
#Demostramos que las columnas "operation" y "country_name" contienen un solo valor unico, por lo que no aportan info relevante
df_val_unicos=pd.DataFrame({"unique_values":df.apply(lambda x : x.unique())})
df_val_unicos["q_unique_values"]=df_val_unicos["unique_values"].apply(lambda x: len(x))
df_val_unicos

,unique_values,q_unique_values
ID de registro,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",121220
Operacion,[sell],1
Tipo de propiedad,"[PH, apartment, house, store]",4
Partido,"[Mataderos, La Plata, Liniers, Centro, Gualegu...",1061
place_with_parent_names,"[|Argentina|Capital Federal|Mataderos|, |Argen...",1164
Pais,[Argentina],1
Provincia,"[Capital Federal, Bs.As. G.B.A. Zona Sur, Buen...",28
geonames_id,"[3430787.0, 3432039.0, 3431333.0, 3435548.0, 3...",647
LatLon,"[-34.6618237,-58.5088387, -34.9038831,-57.9643...",47204
Latitud,"[-34.6618237, -34.9038831, -34.6522615, -34.64...",46496


In [8]:
#Forma alternativa para desmotrar lo anterior
df.Pais.describe()

count        121220
unique            1
top       Argentina
freq         121220
Name: Pais, dtype: object

In [9]:
df.Operacion.describe()

count     121220
unique         1
top         sell
freq      121220
Name: Operacion, dtype: object

In [10]:
#Dropeamos las columnas mencionadas
#Tambien dropeamos las columnas que contienen url's ya que no contienen informacion relevante
df=df.drop(["ID de registro","Operacion","index", "Pais", "image_thumbnail", "properati_url"], axis=1)
df.columns

Index(['Tipo de propiedad', 'Partido', 'place_with_parent_names', 'Provincia',
       'geonames_id', 'LatLon', 'Latitud', 'Longitud', 'Precio Aviso',
       'Moneda Aviso', 'Precio ML', 'Precio USD', 'Superficie Total',
       'Superfecie Cubierta', 'Precio M2 USD', 'Precio M2', 'Piso',
       'Ambientes', 'Expensas', 'Descripcion', 'Título'],
      dtype='object')

In [11]:
#Analizamos la cantidad de nulls por columna en proporcion al total de datos.
#Las columnas con mas campos incompletos se ubican al final de la tabla
df_not_nulls= df.notnull().sum(axis=0) / len(df)
df_not_nulls.round(4).sort_values(ascending = False)

Título                       1.0
Descripcion                  1.0
place_with_parent_names      1.0
Provincia                    1.0
Tipo de propiedad            1.0
Partido                   0.9998
geonames_id               0.8456
Superfecie Cubierta       0.8358
Precio USD                0.8316
Moneda Aviso              0.8316
Precio Aviso              0.8316
Precio ML                 0.8316
Precio M2                 0.7231
Superficie Total          0.6756
Longitud                  0.5747
Latitud                   0.5747
LatLon                    0.5747
Precio M2 USD             0.5661
Ambientes                 0.3909
Expensas                  0.1177
Piso                      0.0652
dtype: float64

In [12]:
#Las columnas de latitu y longitud presentan el mismo porcentaje de valores nulos
#Verificamos si cada columna tiene valores en los mismos registros que las otras
lat_indexes = list(df['Latitud'].notnull().index)
long_indexes = list(df['Longitud'].notnull().index)
latlong_indexes = list(df['LatLon'].notnull().index)

print((lat_indexes==long_indexes),
(lat_indexes==latlong_indexes),
(long_indexes==latlong_indexes))

#las columnas presentan valores en los mismos registros.
#Dropeamos la columna de LatLon (lat-lon)
df=df.drop(["LatLon"], axis=1)
df.columns

True True True


Index(['Tipo de propiedad', 'Partido', 'place_with_parent_names', 'Provincia',
       'geonames_id', 'Latitud', 'Longitud', 'Precio Aviso', 'Moneda Aviso',
       'Precio ML', 'Precio USD', 'Superficie Total', 'Superfecie Cubierta',
       'Precio M2 USD', 'Precio M2', 'Piso', 'Ambientes', 'Expensas',
       'Descripcion', 'Título'],
      dtype='object')

In [13]:
#Notamos que las columnas de precios suelen tener el mismo porcentage de registros con datos.
#Si encontramos un tipo de cambio fijo podriamosmantener unicamante una columna de precio, y eliminar las demas junto con el de tipo de moneda
#Verificamos si cada columna tiene valores en los mismos registros que las otras
Precio_Aviso_indexes = list(df['Precio Aviso'].notnull().index)
Moneda_Aviso_indexes = list(df['Moneda Aviso'].notnull().index)
Precio_ML_indexes = list(df['Precio ML'].notnull().index)
Precio_USD_indexes = list(df['Precio USD'].notnull().index)

print((Precio_Aviso_indexes==Moneda_Aviso_indexes),
(Precio_Aviso_indexes==Precio_ML_indexes),
(Precio_USD_indexes==Precio_Aviso_indexes))

# guardo el tipo de cambio para un analisis posterior
def Tipo_Cambio(arg):
    if (arg["Precio ML"] is None) or (arg["Precio USD"] is None) or (arg["Precio USD"]==0):
        pass
    else:
        TC=arg["Precio ML"]/arg["Precio USD"]
        return TC

df["TC"]=df.apply(Tipo_Cambio, axis=1)

df=df.drop(['Precio Aviso', 'Moneda Aviso', 'Precio ML'], axis=1)
df.columns

True True True


Index(['Tipo de propiedad', 'Partido', 'place_with_parent_names', 'Provincia',
       'geonames_id', 'Latitud', 'Longitud', 'Precio USD', 'Superficie Total',
       'Superfecie Cubierta', 'Precio M2 USD', 'Precio M2', 'Piso',
       'Ambientes', 'Expensas', 'Descripcion', 'Título', 'TC'],
      dtype='object')

In [14]:
#expensas
pattern = "(\$|pesos|dolares|Pesos|Dolares|ARS|USD|PEN|UYU)(\s)*([0-9]+)|([0-9]+)(\s)*(\$|pesos|dolares|Pesos|Dolares|ARS|USD|PEN|UYU)"
regex = re.compile(pattern)

result = df['Descripcion'].apply(lambda x: regex.findall(str(x)))
df_result = pd.DataFrame(result.apply(lambda x : len(x) > 0))

#cantidad de pisos que pueden ser recuperados en la descripcion
expenses_in_descriptions = len(result[df_result['Descripcion']]) / len(df)
expenses_in_descriptions

#Notamos que la cantidad de valores relacionados a precios en la descripcion no es significativa como para salvar la columna
df=df.drop(["Expensas"], axis=1)
df.columns

Index(['Tipo de propiedad', 'Partido', 'place_with_parent_names', 'Provincia',
       'geonames_id', 'Latitud', 'Longitud', 'Precio USD', 'Superficie Total',
       'Superfecie Cubierta', 'Precio M2 USD', 'Precio M2', 'Piso',
       'Ambientes', 'Descripcion', 'Título', 'TC'],
      dtype='object')

In [15]:
# recontruccion de informacion en base a la descripcion
#pisos
#pattern = "piso\s([0-9]+)|([a-z]+)\spiso|piso\s([a-z]+)|([0-9]+)([a-z]+)\spiso"
pattern_1 = "([0-9]+)(\s)*(piso|floor)"
#pattern_2 = "(([a-z]+))(\s)*(piso|floor)"
#pattern_3 = "(piso|floor)(\s)*([0-9]+)"
#pattern_4 = "(piso|floor)(\s)*(([a-z]+))"
#pattern_5 = "planta(\s)*(baja)*([0-9]+)*"
#pattern_6 = "piso|planta|floor"
regex_1 = re.compile(pattern_1)
#regex_2 = re.compile(pattern_2)
#regex_3 = re.compile(pattern_3)
#regex_4 = re.compile(pattern_4)
#regex_5 = re.compile(pattern_5)
#regex_6 = re.compile(pattern_6)

result_1 = df['Descripcion'].apply(lambda x: regex_1.findall(str(x)))
#result_2 = df['Descripcion'].apply(lambda x: regex_2.findall(str(x)))
#result_3 = df['Descripcion'].apply(lambda x: regex_3.findall(str(x)))
#result_4 = df['Descripcion'].apply(lambda x: regex_4.findall(str(x)))
#result_5 = df['Descripcion'].apply(lambda x: regex_5.findall(str(x)))
#result_6 = df['Descripcion'].apply(lambda x: regex_6.findall(str(x)))

df_result_1 = pd.DataFrame(result_1.apply(lambda x : len(x) > 0))
#df_result_2 = pd.DataFrame(result_2.apply(lambda x : len(x) > 0))
#df_result_3 = pd.DataFrame(result_3.apply(lambda x : len(x) > 0))
#df_result_4 = pd.DataFrame(result_4.apply(lambda x : len(x) > 0))
#df_result_5 = pd.DataFrame(result_5.apply(lambda x : len(x) > 0))
#df_result_6 = pd.DataFrame(result_6.apply(lambda x : len(x) > 0))

#cantidad de pisos que pueden ser recuperados en la descripcion
floors_in_descriptions_1 = len(result_1[df_result_1['Descripcion']]) / len(df)
#floors_in_descriptions_2 = len(result_2[df_result_2['Descripcion']]) / len(df)
#floors_in_descriptions_3 = len(result_3[df_result_3['Descripcion']]) / len(df)
#floors_in_descriptions_4 = len(result_4[df_result_4['Descripcion']]) / len(df)
#floors_in_descriptions_5 = len(result_5[df_result_5['Descripcion']]) / len(df)
#floors_in_descriptions_6 = len(result_6[df_result_6['Descripcion']]) / len(df)

# vemos que si sumamos la cantidad de registros que potencialmente podrian recuperarse no llegamos a una cantidad de datos relevantes
print(floors_in_descriptions_1)
#print(floors_in_descriptions_2)
#print(floors_in_descriptions_3)
#print(floors_in_descriptions_4)
#print(floors_in_descriptions_5)
#print(floors_in_descriptions_6) # con este porcentaje podemos notar que las expresiones regulares se estan solapando

df=df.drop(["Piso"], axis=1)
df.columns

0.04898531595446296


Index(['Tipo de propiedad', 'Partido', 'place_with_parent_names', 'Provincia',
       'geonames_id', 'Latitud', 'Longitud', 'Precio USD', 'Superficie Total',
       'Superfecie Cubierta', 'Precio M2 USD', 'Precio M2', 'Ambientes',
       'Descripcion', 'Título', 'TC'],
      dtype='object')

In [16]:
#reconstruccion de la columna de ambientes en base a las columnas titulo y description
pattern = "([0-9]+)(\s)*(amb|AMB|Amb)"
#pattern = "(Mono|mono)(\s)*(amb|AMB|Amb)"
regex = re.compile(pattern)
result_title = df['Título'].apply(lambda x: regex.findall(str(x)))
ambientes_title = result_title.apply(lambda x: x[0][0] if len(x) > 0 else 0)
#ambientes_title = result_title.apply(lambda x: 1 if len(x) > 0 else 0)

df_result_title = pd.DataFrame(ambientes_title.apply(lambda x : x != 0))
df['Ambientes'][df_result_title['Título']] = ambientes_title[df_result_title['Título']]

result_description = df['Descripcion'].apply(lambda x: regex.findall(str(x)))
ambientes_description = result_description.apply(lambda x: x[0][0] if len(x) > 0 else 0)
#ambientes_description = result_description.apply(lambda x: 1 if len(x) > 0 else 0)

df_result_description = pd.DataFrame(ambientes_description.apply(lambda x : x != 0))
df['Ambientes'][df_result_description['Descripcion']] = ambientes_description[df_result_description['Descripcion']]

#analizamos la proporcion de registros en la columna imputada
len(df["Ambientes"][df["Ambientes"].notnull()])/len(df)

C:\Users\rodri\AppData\Local\Temp\ipykernel_1968\3565952944.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.5706979046361986

In [17]:
#construccion de la columna de amenities en base a las columnas titulo y description
pattern = "(pileta|gimnasio|quincho|parrilla|SUM)"
regex = re.compile(pattern, re.IGNORECASE)

#result_title = df['Título'].apply(lambda x: regex.findall(str(x)))
#amenities_title = result_title.apply(lambda x: '|'.join(filter(None, x)) if len(x) > 0 else None)
#df['Amenities'] = amenities_title

#los amenities obtenidos por el campo title se superponen con los obtenidos por el campo description

result_description = df['Descripcion'].apply(lambda x: regex.findall(str(x)))
amenities_description = result_description.apply(lambda x: len(np.unique(list(map(lambda y: y.lower(), filter(None, x))))) if len(x) > 0 else None)
df['Amenities'] = amenities_description

len(df["Amenities"][df["Amenities"].notnull()])/len(df)

0.43419402738821977

In [18]:
#imputamos el precio por m2 en dolares a partir del precio en m2 y el campo TC
def calculatePriceM2USD(x):
  if(np.isnan(x['Precio M2 USD']) and not np.isnan(x['Precio M2'])):
    return x['Precio M2'] / x['TC']
  else:
    return x['Precio M2']

    
df["Precio M2 USD"]=df.apply(calculatePriceM2USD, axis=1)

In [19]:
#analizamos la proporcion de registros en la columna imputada
len(df["Precio M2 USD"][df["Precio M2 USD"].notnull()])/len(df)

0.7231314964527306

In [20]:
#imputamos la superficie total a partir de los campos precio usd y precio m2 usd
def calculateSup(x):
  if(not np.isnan(x['Precio USD']) and not np.isnan(x['Precio M2 USD']) and np.isnan(x['Superficie Total'])):
    return x['Precio USD'] / x['Precio M2 USD']
  else:
    return x['Superficie Total']
    
df["Superficie Total"]=df.apply(calculateSup, axis=1)

In [21]:
#analizamos la proporcion de registros en la columna imputada
len(df["Superficie Total"][df["Superficie Total"].notnull()])/len(df)

0.8760930539514932

In [22]:
#podemos observar que las columnas partido y place_with_parent_names estan relacionadas
#la columna place_with_parent_names contiene una concatenacion de valores que describen
#a la ubicacion del inmueble donde el ultimo valor es el partido.

#imputamos la superficie total a partir de los campos precio usd y precio m2 usd
def FillPlaceNames(x):
    splited_values = x['place_with_parent_names'].split('|')[::-1]
    for value in splited_values:
        if(not (value == '')):
            return value
    return x['Partido']
    
df["Partido"]=df.apply(FillPlaceNames, axis=1)

df=df.drop(["place_with_parent_names", "geonames_id"], axis=1)
df.columns

#analizamos la proporcion de registros en la columna imputada
len(df["Partido"][df["Partido"].notnull()])/len(df)

1.0

In [23]:
# Analizamos la cantidad de nulls por filas en proporcion al total de columnas
#df_not_nulls_per_row = df.notnull().sum(axis = 1) / len(df.columns)

# filtramos por los registros que no cumplan con un cierto porcentaje de datos en sus columnas
#df_not_nulls_per_row_45_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.45])
#df_not_nulls_per_row_50_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.5])
#df_not_nulls_per_row_55_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.55])
#df_not_nulls_per_row_60_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.60])
#df_not_nulls_per_row_65_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.65])
#df_not_nulls_per_row_70_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.70])
#df_not_nulls_per_row_75_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.75])
#df_not_nulls_per_row_80_per  = len(df_not_nulls_per_row[df_not_nulls_per_row < 0.80])

#mostramos la cantidad porcentual de datos que
#print(' 45% de columnas vacias: ' + str(df_not_nulls_per_row_45_per) + 'u  | ' + str(df_not_nulls_per_row_45_per / len(df)) + '% registros\n',
#      '50% de columnas vacias: ' + str(df_not_nulls_per_row_50_per) + 'u  | ' +  str(df_not_nulls_per_row_50_per / len(df)) + '% registros\n',
#      '55% de columnas vacias: ' + str(df_not_nulls_per_row_55_per) + 'u | ' +  str(df_not_nulls_per_row_55_per / len(df)) + '% registros\n',
#      '60% de columnas vacias: ' + str(df_not_nulls_per_row_60_per) + 'u | ' +  str(df_not_nulls_per_row_60_per / len(df)) + '% registros\n',
#      '65% de columnas vacias: ' + str(df_not_nulls_per_row_65_per) + 'u | ' +  str(df_not_nulls_per_row_65_per / len(df)) + '% registros\n',
#      '70% de columnas vacias: ' + str(df_not_nulls_per_row_70_per) + 'u | ' +  str(df_not_nulls_per_row_70_per / len(df)) + '% registros\n',
#      '75% de columnas vacias: ' + str(df_not_nulls_per_row_75_per) + 'u | ' +  str(df_not_nulls_per_row_75_per / len(df)) + '% registros\n',
#      '80% de columnas vacias: ' + str(df_not_nulls_per_row_80_per) + 'u | ' +  str(df_not_nulls_per_row_80_per / len(df)) + '% registros')

In [24]:
#Vemos las caracteristicas generales de las distintas variables del dataset
df.describe()

,Latitud,Longitud,Precio USD,Superficie Total,Superfecie Cubierta,Precio M2 USD,Precio M2,TC,Amenities
count,"69,670.0","69,670.0","100,810.0","106,200.0","101,313.0","87,658.0","87,658.0","100,809.0","52,633.0"
mean,-34.62620984295557,-59.26628952225032,"239,700.5769284793",698.0975802052209,133.05018112186985,"4,891.223394500322","6,912.215704519683",17.644499988455586,1.6770087207645394
std,1.9809362513227842,2.2999223901611723,"391,323.8757320808","10,635.18804762383",724.3514788261274,"13,095.232796524984","28,378.644733892193",4.607770093825213e-08,0.8108591473656144
min,-54.82398489999999,-75.67893099,0.0,0.0,0.0,0.1818445492778271,1.5102040816326532,17.644498249101435,1.0
25%,-34.669065,-58.727040349999996,"89,733.88",55.0,45.0,637.5924509053813,"1,550.0",17.6445,1.0
50%,-34.597985113,-58.4801284,"145,000.0",120.0,75.0,"1,803.2852344315206","2,213.114754098361",17.6445,1.0
75%,-34.441299275,-58.395907675000004,"265,000.0",487.0,150.0,"2,926.829268292683","3,355.5486111111113",17.6445,2.0
max,4.5458426,-53.73332979999999,"46,545,445.0","3,299,521.5000000005","187,000.0","692,046.0","3,999,999.96",17.6445,5.0


In [25]:
#Obervamos si las variables estan correlacionadas
df_corr=df.corr().round(4)
df_corr


,Latitud,Longitud,Precio USD,Superficie Total,Superfecie Cubierta,Precio M2 USD,Precio M2,TC,Amenities
Latitud,1.0,-0.2943,0.0671,0.0073,0.0081,0.095,0.0861,-0.158,0.0173
Longitud,-0.2943,1.0,-0.0261,-0.0089,-0.0095,-0.1019,-0.0878,0.1941,0.0652
Precio USD,0.0671,-0.0261,1.0,0.0724,0.1655,0.0295,0.0561,0.1051,0.122
Superficie Total,0.0073,-0.0089,0.0724,1.0,0.8475,-0.0204,-0.011,0.0137,0.0168
Superfecie Cubierta,0.0081,-0.0095,0.1655,0.8475,1.0,-0.0289,-0.0206,0.0191,0.025
Precio M2 USD,0.095,-0.1019,0.0295,-0.0204,-0.0289,1.0,0.5298,-0.2155,0.0413
Precio M2,0.0861,-0.0878,0.0561,-0.011,-0.0206,0.5298,1.0,-0.1301,0.0199
TC,-0.158,0.1941,0.1051,0.0137,0.0191,-0.2155,-0.1301,1.0,0.0204
Amenities,0.0173,0.0652,0.122,0.0168,0.025,0.0413,0.0199,0.0204,1.0


# Modelado

In [26]:
#verificamos como quedo nuestro modelo despues de la limpieza de datos
df_not_nulls= df.notnull().sum(axis=0) / len(df)
df_not_nulls.round(4).sort_values(ascending = False)

Título                   1.0
Descripcion              1.0
Provincia                1.0
Partido                  1.0
Tipo de propiedad        1.0
Superficie Total      0.8761
Superfecie Cubierta   0.8358
TC                    0.8316
Precio USD            0.8316
Precio M2             0.7231
Precio M2 USD         0.7231
Longitud              0.5747
Latitud               0.5747
Ambientes             0.5707
Amenities             0.4342
dtype: float64

In [27]:
#filtramos el modelo por los inmuebles ubicados en capital federal
state_df = df[df["Provincia"] == 'Capital Federal']
state_df.head(3)

,Tipo de propiedad,Partido,Provincia,Latitud,Longitud,Precio USD,Superficie Total,Superfecie Cubierta,Precio M2 USD,Precio M2,Ambientes,Descripcion,Título,TC,Amenities
0,PH,Mataderos,Capital Federal,-34.6618237,-58.5088387,"62,000.0",55.0,40.0,"1,550.0","1,550.0",2,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,17.6445,nan
2,apartment,Mataderos,Capital Federal,-34.6522615,-58.5229825,"72,000.0",55.0,55.0,"1,309.090909090909","1,309.090909090909",2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,17.6445,nan
3,PH,Liniers,Capital Federal,-34.6477969,-58.5164244,"95,000.0",nan,nan,nan,nan,3,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,17.6445,nan


In [28]:
full_df = state_df[["Tipo de propiedad", "Partido", "Superfecie Cubierta", "Ambientes", "Amenities", "Precio USD"]].isnull().any(axis=1)
full_df[~full_df].size

9184

In [29]:
filtered_df = state_df[["Tipo de propiedad", "Partido", "Superfecie Cubierta","Superficie Total", "Ambientes", "Amenities", "Precio USD","Precio M2 USD"]][~full_df]
filtered_df["Precio M2 USD"]=filtered_df["Precio USD"]/filtered_df["Superficie Total"]
filtered_df.head(10)

,Tipo de propiedad,Partido,Superfecie Cubierta,Superficie Total,Ambientes,Amenities,Precio USD,Precio M2 USD
13,apartment,Palermo Soho,30.0,50.0,3,1.0,"111,700.0","2,234.0"
14,apartment,Palermo Soho,31.0,42.0,1.0,1.0,"147,900.0","3,521.4285714285716"
16,PH,Mataderos,98.0,140.0,4,1.0,"239,000.0","1,707.142857142857"
38,apartment,Las Cañitas,130.0,220.0,3.0,3.0,"480,000.0","2,181.818181818182"
51,house,Boedo,324.0,504.0,6,2.0,"550,000.0","1,091.2698412698412"
60,apartment,Nuñez,50.0,50.0,2,1.0,"173,000.0","3,460.0"
98,apartment,Almagro,40.0,50.0,2,2.0,"109,900.0","2,198.0"
141,house,Floresta,170.0,600.0,3.0,1.0,"219,500.0",365.8333333333333
157,apartment,Balvanera,80.0,"1,411.56",4,1.0,"129,000.0",91.38825129643799
187,PH,Barracas,120.0,156.0,5,2.0,"160,000.0","1,025.6410256410256"


In [30]:
filtered_df["Partido"] = filtered_df["Partido"].replace(['Palermo Chico','Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo'],'Palermo')

In [31]:
filtered_df2=filtered_df[["Tipo de propiedad", "Partido", "Superfecie Cubierta","Superficie Total", "Ambientes", "Amenities","Precio M2 USD","Precio USD"]]
filtered_df2.head(10)

,Tipo de propiedad,Partido,Superfecie Cubierta,Superficie Total,Ambientes,Amenities,Precio M2 USD,Precio USD
13,apartment,Palermo,30.0,50.0,3,1.0,"2,234.0","111,700.0"
14,apartment,Palermo,31.0,42.0,1.0,1.0,"3,521.4285714285716","147,900.0"
16,PH,Mataderos,98.0,140.0,4,1.0,"1,707.142857142857","239,000.0"
38,apartment,Las Cañitas,130.0,220.0,3.0,3.0,"2,181.818181818182","480,000.0"
51,house,Boedo,324.0,504.0,6,2.0,"1,091.2698412698412","550,000.0"
60,apartment,Nuñez,50.0,50.0,2,1.0,"3,460.0","173,000.0"
98,apartment,Almagro,40.0,50.0,2,2.0,"2,198.0","109,900.0"
141,house,Floresta,170.0,600.0,3.0,1.0,365.8333333333333,"219,500.0"
157,apartment,Balvanera,80.0,"1,411.56",4,1.0,91.38825129643799,"129,000.0"
187,PH,Barracas,120.0,156.0,5,2.0,"1,025.6410256410256","160,000.0"


In [32]:
filtered_df2.loc[filtered_df2["Precio M2 USD"]>20000]

,Tipo de propiedad,Partido,Superfecie Cubierta,Superficie Total,Ambientes,Amenities,Precio M2 USD,Precio USD
8396,house,Caballito,1.0,17.6445,4,1.0,"24,936.94919096602","440,000.0"
8924,PH,Saavedra,110.0,10.0,4,1.0,"25,000.0","250,000.0"
25012,apartment,Boedo,37.0,37.0,1,2.0,"32,486.486486486487","1,202,000.0"
25016,apartment,Flores,37.0,37.0,1,1.0,"35,783.78378378379","1,324,000.0"
25031,apartment,Palermo,43.0,50.0,2,2.0,"24,574.0","1,228,700.0"
...,...,...,...,...,...,...,...,...
42575,house,Monte Castro,1.0,17.6445,6,1.0,"23,236.702655218338","410,000.0"
48473,house,Versalles,1.0,17.6445,5,2.0,"28,904.191107710616","510,000.0"
97543,apartment,Villa Luro,40.0,42.0,2,1.0,"32,885.54761904762","1,381,193.0"
97640,apartment,Villa Luro,42.0,45.0,2,1.0,"32,967.53333333333","1,483,539.0"


In [33]:
df=filtered_df2[["Tipo de propiedad", "Partido", "Superfecie Cubierta", "Ambientes", "Amenities","Precio M2 USD"]]
df.shape

(9184, 6)

In [34]:
df.describe()

,Superfecie Cubierta,Amenities,Precio M2 USD
count,"9,184.0","9,184.0","9,184.0"
mean,78.77471689895471,1.7320339721254356,"3,450.991802502786"
std,104.38881244198232,0.7794831553540061,"5,719.534755932943"
min,1.0,1.0,1.315614168679741
25%,37.0,1.0,"1,255.2301255230125"
50%,50.0,2.0,"2,427.3922330097084"
75%,85.0,2.0,"3,240.1465517241377"
max,"5,350.0",5.0,"206,333.33333333334"


In [35]:
#Descartamos interaccion entre los predictores. no existe colinealidad
df.corr()

,Superfecie Cubierta,Amenities,Precio M2 USD
Superfecie Cubierta,1.0,-0.007808228983283671,-0.09425153783435691
Amenities,-0.007808228983283671,1.0,0.07240505058783137
Precio M2 USD,-0.09425153783435691,0.07240505058783137,1.0


In [36]:
dummies=pd.get_dummies(df["Tipo de propiedad"],drop_first=True)
dummies.columns

Index(['apartment', 'house', 'store'], dtype='object')

In [37]:
df_dum_prop=pd.concat([df,dummies],axis='columns')
df_dum_prop.drop(["Tipo de propiedad"], axis=1, inplace=True)
df_dum_prop.head(10)


,Partido,Superfecie Cubierta,Ambientes,Amenities,Precio M2 USD,apartment,house,store
13,Palermo,30.0,3,1.0,"2,234.0",1,0,0
14,Palermo,31.0,1.0,1.0,"3,521.4285714285716",1,0,0
16,Mataderos,98.0,4,1.0,"1,707.142857142857",0,0,0
38,Las Cañitas,130.0,3.0,3.0,"2,181.818181818182",1,0,0
51,Boedo,324.0,6,2.0,"1,091.2698412698412",0,1,0
60,Nuñez,50.0,2,1.0,"3,460.0",1,0,0
98,Almagro,40.0,2,2.0,"2,198.0",1,0,0
141,Floresta,170.0,3.0,1.0,365.8333333333333,0,1,0
157,Balvanera,80.0,4,1.0,91.38825129643799,1,0,0
187,Barracas,120.0,5,2.0,"1,025.6410256410256",0,0,0


In [38]:
dummies=pd.get_dummies(df["Partido"],drop_first=True)
dummies.columns

Index(['Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte',
       'Belgrano', 'Boca', 'Boedo', 'Caballito', 'Capital Federal',
       'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales',
       'Congreso', 'Constitución', 'Flores', 'Floresta', 'Las Cañitas',
       'Liniers', 'Mataderos', 'Monserrat', 'Monte Castro', 'Nuñez', 'Once',
       'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiza', 'Villa del Parque'],
      dtype='object')

In [39]:
df_dum_bar=pd.concat([df_dum_prop,dummies],axis='columns')
df_dum_bar.drop(["Partido"], axis=1, inplace=True)
df_dum_bar.head()

,Superfecie Cubierta,Ambientes,Amenities,Precio M2 USD,apartment,house,store,Agronomía,Almagro,Balvanera,...,Villa Lugano,Villa Luro,Villa Ortuzar,Villa Pueyrredón,Villa Real,Villa Riachuelo,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque
13,30.0,3,1.0,"2,234.0",1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,31.0,1.0,1.0,"3,521.4285714285716",1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,98.0,4,1.0,"1,707.142857142857",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,130.0,3.0,3.0,"2,181.818181818182",1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,324.0,6,2.0,"1,091.2698412698412",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
df_def=df_dum_bar

In [41]:
fig = go.Figure()

fig.add_trace(go.Box(
#    x=df_filtrado_prov["Provincia"],
    y=df["Precio M2 USD"],
    boxmean=True))
fig.update_layout(yaxis_range=[-100,25000], title="Precio M2 USD")
fig.show()

In [42]:
#Version sin outliers
df_def_sinout=df_def.loc[(df_def["Precio M2 USD"]<6199.187) & (df_def["Precio M2 USD"]>1.31)]
df_def_sinout.shape

(8404, 63)

In [43]:
df_def_sinout.describe()

,Superfecie Cubierta,Amenities,Precio M2 USD,apartment,house,store,Agronomía,Almagro,Balvanera,Barracas,...,Villa Lugano,Villa Luro,Villa Ortuzar,Villa Pueyrredón,Villa Real,Villa Riachuelo,Villa Santa Rita,Villa Soldati,Villa Urquiza,Villa del Parque
count,"8,404.0","8,404.0","8,404.0","8,404.0","8,404.0","8,404.0","8,404.0","8,404.0","8,404.0","8,404.0",...,"8,404.0","8,404.0","8,404.0","8,404.0","8,404.0","8,404.0","8,404.0","8,404.0","8,404.0","8,404.0"
mean,79.99226558781533,1.7134697762970015,"2,147.8993014397665",0.8620894811994289,0.0679438362684436,0.002974773917182294,0.00202284626368396,0.026772965254640648,0.0101142313184198,0.010828177058543552,...,0.004521656354117087,0.014278914802475012,0.003926701570680628,0.006544502617801047,0.0011899095668729176,0.00023798191337458352,0.002974773917182294,0.00023798191337458352,0.0679438362684436,0.023798191337458353
std,106.05450071604963,0.7998598012730925,"1,401.1131213113135",0.3448265596888179,0.25166447439939865,0.05446354374747457,0.04493322377457814,0.1614288524524237,0.10006560405351135,0.1034997695424088,...,0.06709505678567104,0.11864527969887227,0.06254396891865757,0.08063774448443743,0.034476587982651866,0.015425744465311425,0.05446354374747457,0.015425744465311426,0.2516644743993986,0.152429006865689
min,1.0,1.0,1.315614168679741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,38.0,1.0,"1,125.4878048780488",1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,53.0,2.0,"2,314.9074074074074",1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,87.0,2.0,"2,937.240135135135",1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,"5,350.0",5.0,"6,199.1869918699185",1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [44]:
fig = go.Figure()

fig.add_trace(go.Box(
#    x=df_filtrado_prov["Provincia"],
    y=df_def_sinout["Precio M2 USD"],
    boxmean=True))
fig.update_layout(yaxis_range=[-100,7000], title="Precio M2 USD")
fig.show()

In [45]:
features=['Superfecie Cubierta',
 'Ambientes',
 'Amenities',
 'apartment',
 'house',
 'store',
 'Agronomía',
 'Almagro',
 'Balvanera',
 'Barracas',
 'Barrio Norte',
 'Belgrano',
 'Boca',
 'Boedo',
 'Caballito',
 'Capital Federal',
 'Centro / Microcentro',
 'Chacarita',
 'Coghlan',
 'Colegiales',
 'Congreso',
 'Constitución',
 'Flores',
 'Floresta',
 'Las Cañitas',
 'Liniers',
 'Mataderos',
 'Monserrat',
 'Monte Castro',
 'Nuñez',
 'Once',
 'Palermo',
 'Parque Avellaneda',
 'Parque Centenario',
 'Parque Chacabuco',
 'Parque Chas',
 'Parque Patricios',
 'Paternal',
 'Pompeya',
 'Puerto Madero',
 'Recoleta',
 'Retiro',
 'Saavedra',
 'San Cristobal',
 'San Nicolás',
 'San Telmo',
 'Tribunales',
 'Velez Sarsfield',
 'Versalles',
 'Villa Crespo',
 'Villa Devoto',
 'Villa General Mitre',
 'Villa Lugano',
 'Villa Luro',
 'Villa Ortuzar',
 'Villa Pueyrredón',
 'Villa Real',
 'Villa Riachuelo',
 'Villa Santa Rita',
 'Villa Soldati',
 'Villa Urquiza',
 'Villa del Parque']

In [50]:
def prom(x):
    x=x.tolist()
    suma=0
    for i in x:
        suma= suma+i
    prom=suma/len(x)
    return prom

In [47]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC


#Separamos los sets de aprendizaje y de testeo
X = df_def[features]
y = df_def["Precio M2 USD"]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=123)

In [48]:
Xtrain_sklearn = Xtrain[features]

model = LinearRegression(fit_intercept=True)

model.fit(Xtrain_sklearn, ytrain)

coef=model.coef_
#coef=coef.tolist()
print("Coeficientes Beta: ",list(map(lambda x:f"{x:8f}",coef)))

#print(model.intercept_)

n = len(ytrain)
p = 62
ypred_train = model.predict(Xtrain_sklearn)
r2 = metrics.r2_score(ytrain, ypred_train)
r2_ajustado = 1 - (1 - r2)*((n-1)/(n - p - 1))

print("R2: ", metrics.r2_score(ytrain, ypred_train))
print("R2 ajustado: ", r2_ajustado)
print('Mean Absolute Error:', metrics.mean_absolute_error(ytrain, ypred_train))
print('Mean Squared Error:', metrics.mean_squared_error(ytrain, ypred_train))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytrain, ypred_train)))

Coeficientes Beta:  ['-1.859242', '-0.058760', '262.142196', '1483.165925', '585.187433', '815.731114', '-481.242456', '1824.436991', '-428.108432', '350.081805', '1328.776766', '547.281986', '1288.846625', '9330.901250', '155.995661', '-411.284505', '653.054457', '-295.328365', '104.471611', '14.057776', '1768.870804', '-438.268000', '-333.429333', '-385.903190', '1357.220464', '-309.674203', '28.479372', '-1436.917883', '736.571637', '760.885478', '-690.182120', '915.202876', '-565.744732', '-331.625334', '-518.709591', '-1913.756257', '-137.233972', '357.071052', '-771.083413', '2902.800392', '659.746008', '928.185405', '549.259173', '11257.599189', '-111.521988', '-125.578322', '710.546295', '199.651587', '1832.328075', '40.316693', '586.054914', '-49.174602', '-669.101517', '-135.309054', '228.025956', '-612.120965', '-355.049783', '-147.136548', '2013.884145', '-848.118509', '-52.509067', '240.906968']
R2:  0.34336259309494477
R2 ajustado:  0.33739753533258376
Mean Absolute Error

In [54]:
#regresion sin regularizacion
scores = cross_validate(model, Xtrain, ytrain, cv=5,
                        scoring=('r2', 'neg_mean_squared_error'),
                        return_train_score=True)

print(f"R2: {scores['train_r2']}")
print(f"Prom R2: {prom(scores['train_r2'])}")

R2: [0.24598506 0.2426416  0.24696912 0.2530085  0.25274316]
Prom R2: 0.2482694873622941


In [52]:
#Modelo sin outliers
#Separamos los sets de aprendizaje y de testeo
X = df_def_sinout[features]
y = df_def_sinout["Precio M2 USD"]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=123)

In [55]:
Xtrain_sklearn = Xtrain[features]

model = LinearRegression(fit_intercept=True)

model.fit(Xtrain_sklearn, ytrain)

coef=model.coef_
#coef=coef.tolist()
print("Coeficientes Beta: ",list(map(lambda x:f"{x:8f}",coef)))

#print(model.intercept_)

n = len(ytrain)
p = 62
ypred_train = model.predict(Xtrain_sklearn)
r2 = metrics.r2_score(ytrain, ypred_train)
r2_ajustado = 1 - (1 - r2)*((n-1)/(n - p - 1))

print("R2: ", metrics.r2_score(ytrain, ypred_train))
print("R2 ajustado: ", r2_ajustado)
print('Mean Absolute Error:', metrics.mean_absolute_error(ytrain, ypred_train))
print('Mean Squared Error:', metrics.mean_squared_error(ytrain, ypred_train))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytrain, ypred_train)))

Coeficientes Beta:  ['-0.632210', '-0.049536', '115.076159', '766.371882', '-11.324444', '907.291486', '-674.956906', '-236.299675', '-304.592912', '-202.433036', '1212.746211', '496.135668', '-634.531213', '-86.105126', '91.083917', '-302.575822', '717.843395', '-171.019665', '136.424103', '-87.225172', '-664.653413', '-354.008371', '-484.744648', '-646.948368', '1427.160274', '-456.139528', '-190.055271', '-1249.967593', '-639.597320', '532.621878', '-622.563154', '793.462222', '-1135.651161', '-138.634090', '-433.867042', '-1842.071075', '-141.151948', '-124.507967', '-839.680742', '1809.421006', '671.825598', '806.885178', '41.492439', '-16.742668', '-150.865344', '64.589579', '1060.610933', '-126.034142', '-551.430769', '114.289053', '-192.350894', '-317.736148', '-902.798761', '-316.687188', '265.319182', '-767.346585', '-840.702057', '-420.262194', '-822.868840', '-508.077007', '8.972621', '210.041980']
R2:  0.24677365437251508
R2 ajustado:  0.23928967465634454
Mean Absolute Err

In [56]:
#regresion sin regularizacion
scores = cross_validate(model, Xtrain, ytrain, cv=5,
                        scoring=('r2', 'neg_mean_squared_error'),
                        return_train_score=True)

print(f"R2: {scores['train_r2']}")
print(f"Prom R2: {prom(scores['train_r2'])}")

R2: [0.24598506 0.2426416  0.24696912 0.2530085  0.25274316]
Prom R2: 0.2482694873622941


In [57]:
ridge = linear_model.Ridge()
scores = cross_validate(ridge, Xtrain, ytrain, cv=5,
                        scoring=('r2', 'neg_mean_squared_error'),
                        return_train_score=True)

print(f"R2: {scores['train_r2']}")
print(f"Prom R2: {prom(scores['train_r2'])}")


R2: [0.24584166 0.24253282 0.24685449 0.25282547 0.25259886]
Prom R2: 0.24813066147025742


In [58]:
lasso = linear_model.Lasso()
scores = cross_validate(lasso, Xtrain, ytrain, cv=5,
                        scoring=('r2', 'neg_mean_squared_error'),
                        return_train_score=True)

print(f"R2: {scores['train_r2']}")
print(f"Prom R2: {prom(scores['train_r2'])}")

R2: [0.24151346 0.23874028 0.24229791 0.24829055 0.24847505]
Prom R2: 0.2438634490045684


In [ ]:
target= df[["Precio M2 USD"]]
type(target)

In [ ]:
# verificamos que variables van a requerir de algun tipo de codificacion
# debatimos sobre si quedarnos unicamente con los registros de palermo ya que existe una gran proporcion de ellos en el dataset

print(np.unique(filtered_df["Tipo de propiedad"]))
print(np.unique(filtered_df["Partido"]))

In [ ]:
#podemos observar que para las features seleccionadas, casi el 90% de los datos pertenece a apartments.
# podemos qudarnos solo con los registros de apartments o buscar registros para los otros tipos de propiedades.
filtered_df['Tipo de propiedad'].value_counts()

In [ ]:
#nos quedamos solo con los registros de appartments
filtered_df = filtered_df[filtered_df["Tipo de propiedad"] == 'apartment']
filtered_df=filtered_df.drop(["Tipo de propiedad"], axis=1)

In [ ]:
#vemos que existen muchos barrios que se llaman palermo.
filtered_df['Partido'].value_counts()

In [ ]:
#para evitar el solapamiento de los registros pertenecientes a alguna de las zonas de palermo, unificamos esos barrios en uno solo
filtered_df["Partido"] = filtered_df["Partido"].replace(['Palermo Chico','Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo'],'Palermo')

In [ ]:
#definimos nuestro dataframe
#utilizamos todos los registros que hay en el dataset filtrado
df_model = filtered_df
df_model

In [ ]:
#defino variables dummies para la columna de partido
dummies = pd.get_dummies(df_model['Partido'],drop_first=True)
dummies

In [ ]:
df_model = pd.concat([df_model,dummies],axis=1)
df_model

In [ ]:
df_model.columns

In [ ]:
X = df_model[['Superfecie Cubierta', 'Ambientes', 'Amenities', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas',
       'Barrio Norte', 'Belgrano', 'Boca', 'Boedo', 'Caballito',
       'Capital Federal', 'Centro / Microcentro', 'Chacarita', 'Coghlan',
       'Colegiales', 'Congreso', 'Constitución', 'Flores', 'Floresta',
       'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat', 'Monte Castro',
       'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario',
       'Parque Chacabuco', 'Parque Chas', 'Parque Patricios', 'Paternal',
       'Pompeya', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra',
       'San Cristobal', 'San Nicolás', 'San Telmo', 'Tribunales',
       'Velez Sarsfield', 'Versalles', 'Villa Crespo', 'Villa Devoto',
       'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar',
       'Villa Pueyrredón', 'Villa Real', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiza', 'Villa del Parque']]
y = df_model['Precio USD']
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

In [ ]:
y_pred

# Analisis Visual de los datos

In [ ]:
c

In [ ]:
#Distribucion del precio del m2 en USD de la totalidad de los datos
fig = px.histogram(df, x="Precio M2 USD", range_x=[0,10000], title="Distribución Precio M2 USD")
#a continuacion detallamos el tamaño de los intervalos
fig.update_traces(xbins=dict(
        start=0.0,
        end=50000000,
        size=500,
    ))
#agregamos la media
fig.add_vline(x=2160.08, line_color="green")
fig.show()

In [ ]:
#Mayor correlacion con Precio del M2 y Superfecie, basicamente pxq.
df_corr_p=df_corr.loc[(np.absolute(df_corr["Precio USD"])>0.3)|(np.absolute(df_corr["Precio USD"])>0.3)][["Precio USD","Precio M2 USD"]]
df_corr_p.loc[np.absolute(df_corr_p["Precio M2 USD"])<0.3,["Precio M2 USD"]]="-"
df_corr_p

In [ ]:
#Relacion entre Superficie Total y Precio USD
fig = px.scatter(x=df["Superficie Total"], y=df["Precio USD"], trendline="ols", trendline_color_override="red",title="Correlacion Precio USD/Superficie Total", 
                 labels={
                     "x": "Superficie Total",
                     "y": "Precio USD",
                 })
fig.update_layout(yaxis_range=[50,1000000],xaxis_range=[0,1000])
fig.show()

In [ ]:
#Relacion entre Precio M2 USD y Precio USD
fig = px.scatter(x=df["Precio M2 USD"], y=df["Precio USD"], trendline="ols", trendline_color_override="red", title="Correlacion Precio USD/Precio M2 USD", 
                 labels={
                     "x": "Precio M2 USD",
                     "y": "Precio USD",
                 })
fig.update_layout(yaxis_range=[50,1000000],xaxis_range=[0,2000])
fig.show()

In [ ]:
df.sort_values(by="Precio USD",ascending=False)[["Provincia","Precio USD"]]

In [ ]:
df["Precio USD"].describe()       

In [ ]:
#Analizamos como se distribuyen las ventas geograficamente
df_plt=pd.DataFrame({"Provincia":df.Provincia.value_counts().index,
                     "Cantidad Operaciones":df.Provincia.value_counts(),
                     "Porcentaje":((df.Provincia.value_counts()/len(df))*100).round(2)
                     })

#Creamos una funcion para englobar a las provincias con participacion menor al 1% en una nueva categoria llamada "otros".
def Reclasificador(arg):
    if arg["Porcentaje"]>1:
        return arg["Provincia"]
    else:
        return "otros"
    
#Nueva columna que contiene los elementos reclasificados, si tienen participacion mayor al 1% conservan su nombre.
df_plt["Provincias_recla"]=df_plt.apply(Reclasificador, axis=1)

df_plt.head(9)

In [ ]:
fig = px.pie(df_plt, values='Porcentaje', names="Provincias_recla", color_discrete_sequence=px.colors.sequential.RdBu, title="Participación de las provincias en las de ventas")
fig.show()

In [ ]:
#Analizamos con un box plot los precios en USD en las principales provincias 
df_filtrado_prov=df.loc[((df.Provincia=="Capital Federal")|(df.Provincia=="Bs.As. G.B.A. Zona Norte")|(df.Provincia=="Bs.As. G.B.A. Zona Sur")|(df.Provincia=="Córdoba")|(df.Provincia=="Santa Fe")|(df.Provincia=="Buenos Aires Costa Atlántica")|(df.Provincia=="Buenos Aires Interior")|(df.Provincia=="Bs.As. G.B.A. Zona Oeste"))]

fig = go.Figure()

fig.add_trace(go.Box(
    x=df_filtrado_prov["Provincia"],
    y=df_filtrado_prov["Precio USD"],
    boxmean=True))
fig.add_hline(y=239700.57, line_dash="dash", line_color="green")
fig.update_layout(yaxis_range=[-10000,1000000], title="Boxplots de Precio USD por provincia")
fig.show()

In [ ]:
#Analizamos con un box plot los precios del M2 en USD en las principales provincias 
fig = go.Figure()

fig.add_trace(go.Box(
    x=df_filtrado_prov["Provincia"],
    y=df_filtrado_prov["Precio M2 USD"],
    boxmean=True))

fig.add_hline(y=2160.09, line_dash="dash", line_color="green")
fig.update_layout(yaxis_range=[-1000,6000], title="Boxplots de Precio M2 USD por provincia")

In [ ]:
#Observamos la Superfecie Total promedio de los inmuebles por Provincia
df_filtrado_prov=df_filtrado_prov.groupby(by=["Provincia"]).mean()
df_filtrado_prov.sort_values(by="Superficie Total",ascending=False)["Superficie Total"]
df_filtrado_prov.index


In [ ]:
#Superficie Total Media por Provincia
fig = px.bar(df_filtrado_prov, x=df_filtrado_prov.index, y=df_filtrado_prov["Superficie Total"], title="Tamaño medio de los inmuebles por provincia")
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
#Conclusion: aunque los precios en capital son mas bajos, el m2 es notablemente mas caro comparado con barrio norte.
#Podriamos infererir que esto se debe a que los inmuebles en CABA son mas pequeños que en Barrio Norte donde en general
#son mas caros

In [ ]:
#df[["Precio USD","TC"]]

In [ ]:
#Si redondeamos todos los tc obtenidos a 2 decimales, vemos que todas las operaciones se cerraron al mismo tc
#df.TC.round(2).unique()

In [ ]:
#Conociendo este dato podemos buscar en la serie historia del tc para ver aproximadamente en que fecha se vendieron los inmuebles.
#path2=r"C:\Users\rodri\Desktop\Data Science\Datos históricos USD_ARS.csv"

#df_tc=pd.read_csv(path2)
#df_tc=df_tc[["Fecha","Apertura"]]
#df_tc["Apertura"]=df_tc["Apertura"].apply(lambda x: float(x.replace(',','.')))
#df_tc["Apertura"]=df_tc['Apertura'].round(2)
#df_tc

In [ ]:
#Vemos que la venta se efectuo en Noviembre del 2017 aprox.
#TC=17.64
#df_tc.loc[df_tc["Apertura"]==TC]